In [111]:
import pdfminer, pdfminer.high_level
import os, re
import pandas as pd

In [112]:
# if not os.path.exists('done'):
#     os.makedirs('done')

In [113]:
num_pages = 0
text = ''
file_names = []
for file in os.listdir('.'):
    if file.endswith('.pdf'):
        with open(file, 'rb') as f:
            text += pdfminer.high_level.extract_text(f)
        current_num_pages = len(list(pdfminer.high_level.extract_pages(file)))
        num_pages += current_num_pages
        file_names.extend([file] * current_num_pages)
        # os.rename(file, 'done/' + file)

pages = text.split('\x0c')

In [114]:
pages
pages = [page for page in pages if page]

In [115]:
pages

['NUMER FAKTURY |\n155430471Z\n\nDATA TRANSAKCJI\n2024.07.24\n\n SZÁMLASZÁM\n\n|\n\n TELJESÍTÉS DÁTUMA\n\nDATA WYSTAWIENIA |\n2024.07.25\n\n SZÁMLA KELTE\n\n AZONOSÍTÓ\n\nPNR |\nSIQ15Q\n\nTERMIN PŁATNOŚCI\n2024.07.24\n\n|\n\n FIZETÉSI HATÁRIDŐ\n\nSTRONA |\n\n OLDAL 1 / 1\n\nNAZWA SPRZEDAWCY |\nWizz Air Hungary Légiközlekedési Zrt.\n\n SZÁLLÍTÓ NÉV\n\nODBIORCA |\nOREX TRAVEL Sp. Z o.o.\n\n VEVŐ NÉV\n\n CÍM\n\nADRES |\nBudapest\nLechner Ödön fasor 6.\n1095\n\n CÍM\n\nADRES |\nWarszawa\nAleje Jerozolimskie 132\n02-305\nPL\n\nEU VAT NUMER |\nHU26648525\n\n UNIÓS ADÓSZÁM\n\nNIP ODBIORCY |\n9542736185\n\n VEVŐ ADÓSZÁM\n\nVAT NUMER |\n26648525-2-44\n\n ADÓSZÁM\n\nILOŚĆ\n\nMENNY.\n\n1,00\n\nJEDNOSTKA MIARY\n\nCENA JEDN.\n\nMÉRTÉKEGYSÉG\n\nEGYSÉGÁR\n\nVAT%\n\nÁFA%\n\nKWOTA NETTO\n\nVAT\n\nKWOTA BRUTTO\n\nWALUTA\n\nÉRTÉKE ÁFA NÉLKÜL\n\nÁTHÁRÍTOTT ÁFA\n\nÉRTÉKE ÁFÁVAL\n\nPÉNZNEM\n\nSztuk / db\n\n-40,00\n\n0.00 %\n\n-40,00\n\n0,00\n\n-40,00\n\nEUR\n\nRODZAJ USŁUGI\nBilet lotniczy / Repülőjegy (KTW

In [116]:
NUMER = r'NUMER FAKTURY.*?\n(.*?)\n'
DATA = r'DATA WYSTAWIENIA.*?\n(.*?)\n'
KWOTA = re.compile(r'PODSUMOWANIE.*WALUTA.*\n(.*?\,.*?)\n.*?\,.*EUR', re.DOTALL)

In [117]:
faktury = [['numer', 'data', 'kwota']]
for page in pages:
    numer = re.search(NUMER, page).group(1)
    data = re.search(DATA, page).group(1)
    kwota = re.search(KWOTA, page).group(1)
    faktury.append([numer, data, kwota])

In [118]:
faktury

[['numer', 'data', 'kwota'],
 ['155430471Z', '2024.07.25', '-40,00'],
 ['155431244Z', '2024.07.25', '-4,00'],
 ['155431721Z', '2024.07.25', '-12,00'],
 ['155431801Z', '2024.07.25', '-60,00']]

In [119]:
faktury = pd.DataFrame(faktury[1:], columns=faktury[0])

In [110]:
faktury.to_excel('faktury.xlsx', index=False)